In [1]:
# Download Stanford dog breed dataset
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
!tar -xvf images.tar

In [2]:
!ls; pwd

In [3]:
!cd /kaggle/working/Images;ls

In [4]:
import os
import numpy as np
import pandas as pd


path_list = []

for dirname, _, filenames in os.walk('/kaggle/working/Images'):
    for filename in filenames:
        print('file_path: ', os.path.join(dirname, filename))
        path_list.append(os.path.join(dirname, filename))

In [5]:
# Based on the directory and file name of the image files, 
# the absolute path and label of the image, which are meta-information, are created as DataFrame.

# - label value: Dog Breed Dataset Sub Direcory

start_pos = '/kaggle/working/Images/n02109961-Eskimo_dog/n02109961_12338.jpg'.find('/', 20)
print(start_pos)

end_pos = '/kaggle/working/Images/n02109961-Eskimo_dog/n02109961_12338.jpg'.rfind('/')
print(end_pos)

imsi_breed = '/kaggle/working/Images/n02109961-Eskimo_dog/n02109961_12338.jpg'[start_pos+1:end_pos]
print(imsi_breed)

# Dog Breed Label Name
imsi_breed[imsi_breed.find('-')+1:]

In [6]:
IMAGE_DIR = '/kaggle/working/Images'

In [7]:
def make_dogbreed_dataframe(image_dir=IMAGE_DIR):
    paths = []
    labels = []
    for dirname, _, filenames in os.walk(image_dir):
        for filename in filenames:
            if '.jpg' in filename:
                file_path = dirname + '/' + filename
                paths.append(file_path)
                start_pos = file_path.find('/', 20)
                end_pos = file_path.rfind('/')
                imsi_breed = file_path[start_pos+1:end_pos]
                breed = imsi_breed[imsi_breed.find('-')+1:]
                labels.append(breed)
    
    data_df = pd.DataFrame(
        {
            'path': paths,
            'label': labels
        }
    )
    
    return data_df

In [8]:
pd.set_option('display.max_colwidth', 200)
data_df = make_dogbreed_dataframe()

print('data_df shape: ', data_df.shape)

In [9]:
data_df.head()

In [10]:
# Check the individual distribution of Dog Breed
data_df['label'].value_counts()

In [11]:
# Check the number of cases by bread in the form of a bar graph

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


plt.figure(figsize=(26, 4))

sns.countplot(data=data_df, x='label')
plt.xticks(rotation=90)

In [12]:
# View Dog Breed's Image

import cv2


def show_grid_images(image_path_list, ncols=6, title=None):
    figure, axs = plt.subplots(figsize=(22, 4), nrows=1, ncols=ncols)
    for i in range(ncols):
        image = cv2.cvtColor(cv2.imread(image_path_list[i]), cv2.COLOR_BGR2RGB)
        axs[i].imshow(image)
        axs[i].set_title(title)

In [13]:
breed_image_list_01 = data_df[data_df['label']=='komondor']['path'].iloc[:6].tolist()

show_grid_images(breed_image_list_01, ncols=6, title='komondor')

In [14]:
breed_image_list_02 = data_df[data_df['label']=='dingo']['path'].iloc[:6].tolist()

show_grid_images(breed_image_list_02, ncols=6, title='dingo')

In [15]:
breed_image_list_03 = data_df[data_df['label']=='dhole']['path'].iloc[:6].tolist()

show_grid_images(breed_image_list_03, ncols=6, title='dhole')

In [16]:
breed_list = data_df['label'].value_counts().index.tolist()
print(breed_list)

print(len(breed_list))

In [17]:
for cnt, breed in enumerate(breed_list):
    breed_image_list = data_df[data_df['label']==breed]['path'].iloc[:6].tolist()
    show_grid_images(breed_image_list, ncols=6, title=breed)
    if cnt == 10:
        break

In [18]:
# Fix the image size at 224x224 and look at the image applied with Augmentation

import albumentations as A


imsi_augmentor = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),
    A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5)
])

In [19]:
def show_grid_images(image_path_list, augmentor=None, ncols=4, title=None):
    figure, axs = plt.subplots(figsize=(22, 4), nrows=1, ncols=ncols)
    for i in range(ncols):
        image = cv2.cvtColor(cv2.imread(image_path_list[i]), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224)) # Resize!!!!!!
        if augmentor is not None:
            image = augmentor(image=image)['image']
        axs[i].imshow(image)
        axs[i].axis('off')
        axs[i].set_title(title)

In [20]:
for cnt, breed in enumerate(breed_list):
    breed_image_list = data_df[data_df['label']==breed]['path'].iloc[:6].tolist()
    show_grid_images(breed_image_list, augmentor=None, ncols=6, title=breed)
    show_grid_images(breed_image_list, augmentor=imsi_augmentor, ncols=6, title=breed)
    if cnt == 3:
        break

In [21]:
# Seperate the entire DataFrame into a DataFrame for learning and testing
# Learning DataFrame is seperated again for learning and verification
# - use train_test_split() to allocate 40% of the total to test data
# - configure equal allocation for each breed label with the stratify factor

from sklearn.model_selection import train_test_split

# (Train+Validation/Test) Split
train_df, test_df = train_test_split(data_df, test_size=0.4, stratify=data_df['label'], random_state=2021)
print('train_df shape: ', train_df.shape)
print('test_df shape: ', test_df.shape)

In [22]:
print(train_df['label'].value_counts()/train_df.shape[0])

In [23]:
plt.figure(figsize=(26, 4))

sns.countplot(data=train_df, x='label')
plt.xticks(rotation=90)

In [24]:
print(test_df['label'].value_counts()/test_df.shape[0])

In [25]:
plt.figure(figsize=(26, 4))

sns.countplot(data=test_df, x='label')
plt.xticks(rotation=90)

In [26]:
# Train/Validation Split

train_df.head()

In [27]:
# Convert Dataframe to numpy array

train_path = train_df['path'].values
train_label = pd.get_dummies(train_df['label']).values # one-hot encoding

print(train_path)
print(train_label)

In [28]:
tr_path, val_path, tr_label, val_label = train_test_split(
    train_path,
    train_label,
    stratify=train_label,
    test_size=0.2,
    random_state=2021
)

In [29]:
print('train path shape: ', tr_path.shape)
print('train label shape: ', tr_label.shape)
print('validation path shape: ', val_path.shape)
print('validation label shape: ', val_label.shape)

In [48]:
# Make Function as above

def get_train_valid(train_df, valid_size=0.2, random_state=2021):
    train_path = train_df['path'].values
    train_label = pd.get_dummies(train_df['label']).values
    
    tr_path, val_path, tr_label, val_label = train_test_split(train_path, train_label, test_size=valid_size, random_state=random_state)
    
    print('train path shape: ', tr_path.shape)
    print('train label shape: ', tr_label.shape)
    print('validation path shape: ', val_path.shape)
    print('validation label shape: ', val_label.shape)
    
    return tr_path, val_path, tr_label, val_label

In [54]:
# Sequence-based Dataset

from tensorflow.keras.utils import Sequence
import sklearn
import cv2


BATCH_SIZE = 64
IMAGE_SIZE = 224

# image_filenames, labels(one-hot encoding) -> numpy array
class Dog_Breed_Dataset(Sequence):
    def __init__(self, image_filenames, labels, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                augmentor=None, shuffle=False, pre_func=None):
        '''
        :parameters:
        image_filenames: the absolute paths of files loaded image as opencv
        labels: image label
        batch_size: __getitem__(self, index) -> take data batch nums as invoking that function
        augmentor: albumentations 
        shuffle: In case of train data, shffule or not shuffle on the end of epoch
        '''
        
        self.image_filenames = image_filenames
        self.labels = labels
        self.image_size = image_size
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = pre_func
        
        # If Train Data
        self.shuffle = shuffle
        if self.shuffle:
            # self.on_epoch_end()
            pass
        
    def __len__(self):
        return int(np.ceil(len(self.labels) / self.batch_size))

    def __getitem__(self, index):
        image_name_batch = self.image_filenames[index*self.batch_size:(index+1)*self.batch_size]
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]

        image_batch = np.zeros((image_name_batch.shape[0], self.image_size[0], self.image_size[1], 3), dtype='float32')

        for image_index in range(image_name_batch.shape[0]):
            image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]), cv2.COLOR_BGR2RGB)
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
            image = cv2.resize(image, (self.image_size[0], self.image_size[1]))
            if self.pre_func is not None:
                image = self.pre_func(image)

            image_batch[image_index] = image

        return image_batch, label_batch
    
    def on_epoch_end(self):
        if (self.shuffle):
            self.image_filenames, self.labels = sklearn.utils.shuffle(self.image_filenames, self.labels)
        else:
            pass

In [55]:
# Create Pretrained Model

from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler
from tensorflow.keras.applications import Xception, ResNet50V2, EfficientNetB0, EfficientNetB1
from tensorflow.keras.applications import MobileNet
import tensorflow as tf

# dog breed 종류는 120가지

def create_model(model_type='xception', in_shape=(224, 224, 3), n_classes=120):
    input_tensor = Input(shape=in_shape)
    if model_type == 'resnet50v2':
        base_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'xception':
        base_model = tf.keras.applications.Xception(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb0':
        base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb1':
        base_model = tf.keras.applications.EfficientNetB1(include_top=False, weights='imagenet', input_tensor=input_tensor)
        
    x = base_model.output  
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)    
    preds = Dense(units=n_classes, activation='softmax')(x)
    model = Model(inputs=input_tensor, outputs=preds)
    
    return model

In [56]:
# Re-Create Augmentation for only Horizontal Flip

import albumentations as A


augmentor_light = A.Compose([
    A.HorizontalFlip(p=0.5)
])

In [58]:
import time
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications.xception import preprocess_input as xcp_preprocess_input


tr_ds = Dog_Breed_Dataset(
    tr_path,
    tr_label,
    image_size=(224, 224),
    batch_size=BATCH_SIZE,
    augmentor=augmentor_light,
    shuffle=True,
    pre_func=xcp_preprocess_input
)

val_ds = Dog_Breed_Dataset(
    val_path,
    val_label,
    image_size=(224, 224),
    batch_size=BATCH_SIZE,
    augmentor=None,
    shuffle=False,
    pre_func=xcp_preprocess_input
)

tr_image_batch = next(iter(tr_ds))[0]
val_image_batch = next(iter(val_ds))[0]

print('train image batch shape: ', tr_image_batch.shape)
print('validation image batch shape: ', val_image_batch.shape)

In [59]:
# Dataset을 64건씩 Fetch 하면서 걸리는 시간 측정
start = time.time()

for val1, val2 in iter(tr_ds):
    end = time.time()
    print(end - start)
    start = end

In [60]:
# Callbacks

rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, mode='min', verbose=1)
ely_cb = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)

In [61]:
# Create Train model Function

def train_model(train_df, config=None):
    
    # Train & Validation Split
    tr_path, val_path, tr_label, val_label = \
    get_train_valid(train_df, valid_size=0.2, random_state=2021)
    
    # Create Dataset based on Sequence
    tr_ds = Dog_Breed_Dataset(
        tr_path,
        tr_label,
        image_size=config.image_size,
        batch_size=config.batch_size,
        augmentor=augmentor_light,
        shuffle=True,
        pre_func=config.pre_func
    )
    
    val_ds = Dog_Breed_Dataset(
        val_path,
        val_label,
        image_size=config.image_size,
        batch_size=config.batch_size,
        augmentor=None,
        shuffle=False,
        pre_func=config.pre_func
    )
    
    print()
    # Debugging for prepared dataset based on sequence
    if config.debug:
        print('####################Debugging for prepared dataset based on sequence####################')
        if hasattr(tr_ds, '__iter__'):
            tr_image_batch = next(iter(tr_ds))[0]
            print("train image batch shape: ", tr_image_batch.shape)
        if hasattr(val_ds, '__iter__'):
            val_image_batch = next(iter(val_ds))[0]
            print("validation image batch shape: ", val_image_batch.shape)
    
    print()
    # Create Model
    print(f'###############{config.model_type}###############')
    model = create_model(model_type=config.model_type)
    model.compile(
        optimizer=Adam(lr=config.initial_lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print()
    # Training
    print('###############Training Start###############')
    history = model.fit(
        tr_ds,
        epochs=config.n_epochs,
        steps_per_epoch=int(np.ceil(tr_path.shape[0]/config.batch_size)),
        validation_data=val_ds,
        validation_steps=int(np.ceil(val_path.shape[0]/config.batch_size)),
        callbacks=config.callbacks,
        verbose=1
    )
    
    return model, history

## Xception

In [62]:
# Config를 이용하여 학습 수행

# - Model: Xception
# - Image Size: (320, 512)
# - Batch Size: 64
# - Initial Value of LR: 0.0001
# - LR Scheduler: Ramp up and Step decay
# - epochs = 30
# - not fine tuning
# - augmentor -> augmentor_01

class Config:
    model_type = 'xception'
    image_size = (224, 224)
    batch_size = 64
    n_epochs = 30
    callbacks = [rlr_cb, ely_cb]
    augmentor = augmentor_light
    pre_func = xcp_preprocess_input
    initial_lr = 0.0001
    debug = True

In [63]:
xception_model, xception_history = train_model(train_df, config=Config)

In [67]:
# Prediction

test_path = test_df['path'].values
test_label = pd.get_dummies(test_df['label']).values

print(test_path)
print(test_label)

In [68]:
# numpy argmax 값으로 ground truth 컬럼에 one-hot encoding 된
# class 값이 1인 인덱스 값으로 Label encoding

test_df['gt_class'] = np.argmax(test_label, axis=1)

test_df.head()

In [70]:
# Create test dataset based on sequence

test_ds = Dog_Breed_Dataset(
    test_path,
    test_label,
    image_size=Config.image_size,
    batch_size=Config.batch_size,
    augmentor=None,
    shuffle=False,
    pre_func=xcp_preprocess_input
)

In [71]:
xception_model.evaluate(test_ds)

In [72]:
# 어떤 Breed가 예측이 많이 틀렸는지 확인
# - 실제 Ground Truth Breed와 예측 Breed가 어떻게 틀려졌는지 확인

predict_result = xception_model.predict(test_ds, steps=int(np.ceil(len(test_label)/Config.batch_size)))
predict_class = np.argmax(predict_result, axis=1)

In [73]:
test_df['xcp_pred_class'] = predict_class

In [74]:
test_df.head()

In [75]:
# 예측이 틀린 데이터 확인
test_df[test_df['gt_class'] != test_df['xcp_pred_class']]

In [76]:
test_df[test_df['gt_class'] != test_df['xcp_pred_class']]['label'].value_counts()

In [78]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(26, 4))
plt.xticks(rotation=90)

wrong_result_df = test_df[test_df['gt_class'] != test_df['xcp_pred_class']]
sns.countplot(data=wrong_result_df, x='label')

In [79]:
def show_grid_imagees(image_path_list, augmentor=None, ncols=4, title=None):
    figure, axs = plt.subplots(figsize=(22, 4), nrows=1, ncols=ncols)
    for i in range(ncols):
        image = cv2.cvtColor(cv2.imread(image_path_list[i]), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        if augmentor is not None:
            image = augmentor(image=image)['image']
        axs[i].imshow(image)
        axs[i].axis('off')
        axs[i].set_title(title)
    
breed_image_list_01 = data_df[data_df['label']=='Siberian_husky']['path'].iloc[:6].tolist()
breed_image_list_02 = data_df[data_df['label']=='Eskimo_dog']['path'].iloc[:6].tolist()

show_grid_images(breed_image_list_01, ncols=6, title='Siberian_husky')
show_grid_images(breed_image_list_02, ncols=6, title='Eskimo_dog')